In [219]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import csv

## Authentication

In [220]:
CLIENT_ID = 'd8415d5149674a91a7e81de017c789d7'
CLIENT_SECRET = '576b0598e8014aeb93f7bc1ab69ac410'

client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Playlists

In [221]:
PLAYLIST_ID_MUSICAL_EXPERIMENT = 'https://open.spotify.com/playlist/3tTo8cKpIQbLJ3MTzgEVjV?si=5729dd4f3bcc41fe'

PLAYLIST_ID_MUSICAL_EXPERIMENT_2 = 'https://open.spotify.com/playlist/1Kzd8jonR1022i8dL8eDda?si=615bed3d56b64ed7'

### Mainstream Genres

In [222]:
MAINSTREAM_GENRES_FILE = '../datasets/mainstream_genres.csv'

mainstream_genres = list()

with open(MAINSTREAM_GENRES_FILE, 'r') as f:
    csvreader = csv.reader(f, delimiter='\n')
    for row in csvreader:
        mainstream_genres.append(row[0])
    
    print(mainstream_genres)

['pop', 'hip hop', 'hip pop', 'rap', 'trap', 'drill', 'rock', 'blues', 'jazz', 'country', 'folk', 'r&b', 'motown', 'soul', 'classical', 'electronica', 'electro', 'electropop', 'disco', 'dance', 'invasion', 'emo', 'punk', 'post-grunge', 'grunge', 'metal', 'metalcore', 'reggae', 'k-pop', 'urbano', 'reggaeton', 'latin', 'sertanejo', 'gospel', 'mpb', 'lo-fi', 'soundtrack', 'worship']


## Get Artist Data

In [223]:
def get_artist_genres(artist_id:str) -> set:
    artist = spotify.artist(artist_id)

    return set(artist['genres'])


def get_related_artists_names(artist_id:str) -> str:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_names = {related_artist['name'] for related_artist in related_artists['artists']}

    return "/".join(related_artists_names)

def get_related_artists_genres(artist_id:str, track_genres:set) -> set:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_genres = set()

    for related_artist in related_artists['artists']:
        genres = set(related_artist['genres'])

        for genre in genres:
            related_artists_genres.add(genre)

        print(related_artists_genres)

    return (related_artists_genres - track_genres)

def get_generic_genres(mainstream_genres:list, track_genres:set, track_related_genres:set) -> str:
    generic_genres = set()

    for track_genre in track_genres:
        for mainstream_genre in mainstream_genres:
            if mainstream_genre in track_genre:
                generic_genres.add(mainstream_genre)

    for track_related_genre in track_related_genres:
        for mainstream_genre in mainstream_genres:
            if mainstream_genre in track_related_genre:
                generic_genres.add(mainstream_genre)

    return "/".join(generic_genres)

## Get Track Data

In [224]:
def get_tracks_id(playlist_id:str) -> list:  
    tracks_id = list()
    playlist_items = spotify.playlist_items(playlist_id)

    for item in playlist_items['items']:
        track = item['track']
        tracks_id.append(track['id'])

    return tracks_id

def get_track_data(track_id:str) -> dict:
    metadata = spotify.track(track_id)
    audio_features = spotify.audio_features(track_id)

    artist_id = metadata['album']['artists'][0]['id']
    track_genres = get_artist_genres(artist_id)
    related_track_genres = get_related_artists_genres(artist_id, track_genres)


    return {
        'id': metadata['id'],
        'name': metadata['name'],
        'album': metadata['album']['name'],
        'artist': metadata['album']['artists'][0]['name'],
        'genres': "/".join(track_genres),
        'related_artists': get_related_artists_names(artist_id),
        'related_genres': "/".join(related_track_genres),
        'generic_genres': get_generic_genres(mainstream_genres,track_genres, related_track_genres),
        'release_date': metadata['album']['release_date'],
        'duration_ms': metadata['duration_ms'],
        'popularity': metadata['popularity'],
        'acousticness': audio_features[0]['acousticness'],
        'danceability': audio_features[0]['danceability'],

        'energy': audio_features[0]['energy'],
        'instrumentalness': audio_features[0]['instrumentalness'],
        'key': audio_features[0]['key'],
        'liveness': audio_features[0]['liveness'],
        'loudness': audio_features[0]['loudness'],
        'mode': audio_features[0]['mode'],
        'speechiness': audio_features[0]['speechiness'],
        'tempo': audio_features[0]['tempo'],
        'time_signature': audio_features[0]['time_signature'],
        'valence': audio_features[0]['valence'],

        
    }

In [225]:
tracks_id = get_tracks_id(PLAYLIST_ID_MUSICAL_EXPERIMENT_2)
    

In [226]:
tracks_data = list()

for track_id in tracks_id:
    time.sleep(.5)
    tracks_data.append(get_track_data(track_id))
    print(tracks_data)


{'album rock', 'singer-songwriter', 'rock', 'hard rock', 'classic rock', 'blues rock', 'british blues', 'psychedelic rock', 'folk rock', 'supergroup'}
{'album rock', 'electric blues', 'singer-songwriter', 'rock', 'hard rock', 'classic rock', 'country rock', 'blues rock', 'british blues', 'blues', 'psychedelic rock', 'folk rock', 'supergroup'}
{'album rock', 'electric blues', 'singer-songwriter', 'rock', 'hard rock', 'classic rock', 'country rock', 'blues rock', 'british blues', 'blues', 'psychedelic rock', 'british invasion', 'folk rock', 'supergroup'}
{'album rock', 'electric blues', 'heartland rock', 'singer-songwriter', 'rock', 'hard rock', 'classic rock', 'country rock', 'blues rock', 'british blues', 'blues', 'psychedelic rock', 'british invasion', 'folk rock', 'supergroup', 'mellow gold'}
{'album rock', 'country rock', 'british invasion', 'electric blues', 'hard rock', 'supergroup', 'classic rock', 'british blues', 'folk rock', 'mellow gold', 'classic canadian rock', 'blues', 'he

In [227]:
df = pd.DataFrame(tracks_data)

In [228]:
df

,id,name,album,artist,genres,related_artists,related_genres,generic_genres,release_date,duration_ms,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0NWPxcsf5vdjdiFUI8NgkP,Hey Joe,Are You Experienced,Jimi Hendrix,album rock/acid rock/rock/proto-metal/classic ...,The Yardbirds/The Doors/Big Brother & The Hold...,protopunk/roots rock/country rock/symphonic ro...,punk/rock/metal/invasion/blues/folk/soul/country,1967-05-12,210160,...,0.768,0.380000,9,0.0244,-5.695,1,0.0377,169.492,4,0.532
1,2MuWTIM3b0YEAskbeeFE1i,Master Of Puppets,Master Of Puppets (Remastered),Metallica,thrash metal/hard rock/rock/metal/old school t...,Danzig/Airbourne/Black Label Society/Pantera/O...,album rock/uk doom metal/german thrash metal/g...,post-grunge/punk/rock/metal/rap/metalcore/grun...,1986-03-03,515386,...,0.836,0.431000,4,0.1530,-9.110,0,0.0353,105.173,4,0.560
2,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,Midnights,Taylor Swift,pop,Harry Styles/One Direction/Ariana Grande/Selen...,canadian contemporary r&b/post-teen pop/viral ...,r&b/rock/dance/hip pop/pop/electropop/electro,2022-10-21,200690,...,0.643,0.000002,4,0.1420,-6.571,1,0.0519,97.008,4,0.533
3,4N0TP4Rmj6QQezWV88ARNJ,Superstition,Talking Book,Stevie Wonder,soul/motown,The Miracles/Bill Withers/The O'Jays/Curtis Ma...,chicago soul/vocal jazz/southern soul/p funk/n...,disco/motown/rock/jazz/blues/soul,1972-10-28,266133,...,0.578,0.021900,1,0.0983,-14.171,1,0.0676,100.625,4,0.884
4,0PUi8C3der9IVNFaksxTiw,Rain,Candlebox,Candlebox,post-grunge/alternative rock/nu metal/alternat...,Chris Cornell/Temple Of The Dog/Blind Melon/Co...,supergroup/funk metal/rock/australian psych/cl...,post-grunge/rock/metal/pop/blues/grunge,1993-07-16,418426,...,0.849,0.000199,9,0.0570,-7.109,1,0.0755,101.886,4,0.346
5,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,Facelift,Alice In Chains,rock/alternative rock/nu metal/alternative met...,Audioslave/Chris Cornell/Velvet Revolver/Templ...,glam metal/post-grunge/industrial metal/punk/f...,post-grunge/punk/rock/metal/pop/rap/blues/grunge,1990-08-01,285200,...,0.791,0.000000,8,0.0969,-7.565,1,0.0453,106.392,4,0.758
6,6MKqed1tCXHRonfqhwPCET,When A Blind Man Cries,Hardwired…To Self-Destruct (Deluxe),Metallica,thrash metal/hard rock/rock/metal/old school t...,Danzig/Airbourne/Black Label Society/Pantera/O...,album rock/uk doom metal/german thrash metal/g...,post-grunge/punk/rock/metal/rap/metalcore/grun...,2016-11-18,275013,...,0.770,0.062700,8,0.1160,-5.783,1,0.0392,134.644,4,0.296
7,50DMG6AvtNzbrxMc62w6ph,Bold as Love,Continuum,John Mayer,singer-songwriter/neo mellow,Amos Lee/Parachute/Five For Fighting/Eric Hutc...,viral pop/charlottesville indie/folk-pop/indie...,post-grunge/punk/k-pop/rock/dance/pop/folk/grunge,2006-09-11,258160,...,0.590,0.002210,1,0.0718,-6.407,1,0.0417,125.897,4,0.400
8,3a1lNhkSLSkpJE4MSHpDu9,Congratulations,Stoney (Deluxe),Post Malone,pop/melodic rap/dfw rap/rap,A$AP Ferg/Arizona Zervas/Desiigner/Bazzi/mike....,gangster rap/rap conscient/r&b/australian hip ...,r&b/hip hop/pop/rap/trap,2016-12-09,220293,...,0.804,0.000000,6,0.2530,-4.183,1,0.0363,123.146,4,0.492


In [229]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

In [230]:
df

,id,name,album,artist,genres,related_artists,related_genres,generic_genres,release_date,duration_ms,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0NWPxcsf5vdjdiFUI8NgkP,Hey Joe,Are You Experienced,Jimi Hendrix,album rock/acid rock/rock/proto-metal/classic ...,The Yardbirds/The Doors/Big Brother & The Hold...,protopunk/roots rock/country rock/symphonic ro...,punk/rock/metal/invasion/blues/folk/soul/country,1967-05-12,210160,...,0.768,0.380000,9,0.0244,-5.695,1,0.0377,169.492,4,0.532
1,2MuWTIM3b0YEAskbeeFE1i,Master Of Puppets,Master Of Puppets (Remastered),Metallica,thrash metal/hard rock/rock/metal/old school t...,Danzig/Airbourne/Black Label Society/Pantera/O...,album rock/uk doom metal/german thrash metal/g...,post-grunge/punk/rock/metal/rap/metalcore/grun...,1986-03-03,515386,...,0.836,0.431000,4,0.1530,-9.110,0,0.0353,105.173,4,0.560
2,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,Midnights,Taylor Swift,pop,Harry Styles/One Direction/Ariana Grande/Selen...,canadian contemporary r&b/post-teen pop/viral ...,r&b/rock/dance/hip pop/pop/electropop/electro,2022-10-21,200690,...,0.643,0.000002,4,0.1420,-6.571,1,0.0519,97.008,4,0.533
3,4N0TP4Rmj6QQezWV88ARNJ,Superstition,Talking Book,Stevie Wonder,soul/motown,The Miracles/Bill Withers/The O'Jays/Curtis Ma...,chicago soul/vocal jazz/southern soul/p funk/n...,disco/motown/rock/jazz/blues/soul,1972-10-28,266133,...,0.578,0.021900,1,0.0983,-14.171,1,0.0676,100.625,4,0.884
4,0PUi8C3der9IVNFaksxTiw,Rain,Candlebox,Candlebox,post-grunge/alternative rock/nu metal/alternat...,Chris Cornell/Temple Of The Dog/Blind Melon/Co...,supergroup/funk metal/rock/australian psych/cl...,post-grunge/rock/metal/pop/blues/grunge,1993-07-16,418426,...,0.849,0.000199,9,0.0570,-7.109,1,0.0755,101.886,4,0.346
5,6gZVQvQZOFpzIy3HblJ20F,Man in the Box,Facelift,Alice In Chains,rock/alternative rock/nu metal/alternative met...,Audioslave/Chris Cornell/Velvet Revolver/Templ...,glam metal/post-grunge/industrial metal/punk/f...,post-grunge/punk/rock/metal/pop/rap/blues/grunge,1990-08-01,285200,...,0.791,0.000000,8,0.0969,-7.565,1,0.0453,106.392,4,0.758
6,6MKqed1tCXHRonfqhwPCET,When A Blind Man Cries,Hardwired…To Self-Destruct (Deluxe),Metallica,thrash metal/hard rock/rock/metal/old school t...,Danzig/Airbourne/Black Label Society/Pantera/O...,album rock/uk doom metal/german thrash metal/g...,post-grunge/punk/rock/metal/rap/metalcore/grun...,2016-11-18,275013,...,0.770,0.062700,8,0.1160,-5.783,1,0.0392,134.644,4,0.296
7,50DMG6AvtNzbrxMc62w6ph,Bold as Love,Continuum,John Mayer,singer-songwriter/neo mellow,Amos Lee/Parachute/Five For Fighting/Eric Hutc...,viral pop/charlottesville indie/folk-pop/indie...,post-grunge/punk/k-pop/rock/dance/pop/folk/grunge,2006-09-11,258160,...,0.590,0.002210,1,0.0718,-6.407,1,0.0417,125.897,4,0.400
8,3a1lNhkSLSkpJE4MSHpDu9,Congratulations,Stoney (Deluxe),Post Malone,pop/melodic rap/dfw rap/rap,A$AP Ferg/Arizona Zervas/Desiigner/Bazzi/mike....,gangster rap/rap conscient/r&b/australian hip ...,r&b/hip hop/pop/rap/trap,2016-12-09,220293,...,0.804,0.000000,6,0.2530,-4.183,1,0.0363,123.146,4,0.492


In [231]:
df.to_csv("../datasets/musical_experiment_2.csv", sep=',')